In [1]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

In [2]:
env = gym.make('CartPole-v1')
env.reset()
goal_steps = 500
score_requirement = 60
intial_games = 10000

In [3]:
def play_a_random_game_first():
    for step_index in range(goal_steps):
#         env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("\n")
#         print("info: {}".format(info))
        if done:
            break
    env.reset()

In [4]:
play_a_random_game_first()

Step 0:
action: 1
observation: [-0.02558117  0.20405725 -0.03786342 -0.26767697]
reward: 1.0
done: False


Step 1:
action: 0
observation: [-0.02150003  0.00949555 -0.04321696  0.01282722]
reward: 1.0
done: False


Step 2:
action: 0
observation: [-0.02131012 -0.18498083 -0.04296042  0.29156757]
reward: 1.0
done: False


Step 3:
action: 0
observation: [-0.02500973 -0.37946474 -0.03712907  0.57039779]
reward: 1.0
done: False


Step 4:
action: 0
observation: [-0.03259903 -0.57404686 -0.02572111  0.85115637]
reward: 1.0
done: False


Step 5:
action: 1
observation: [-0.04407997 -0.37858383 -0.00869798  0.55049764]
reward: 1.0
done: False


Step 6:
action: 1
observation: [-0.05165164 -0.18334079  0.00231197  0.25508699]
reward: 1.0
done: False


Step 7:
action: 1
observation: [-0.05531846  0.01174807  0.00741371 -0.03686581]
reward: 1.0
done: False


Step 8:
action: 1
observation: [-0.0550835   0.20676293  0.00667639 -0.32720044]
reward: 1.0
done: False


Step 9:
action: 1
observation: [-5.09

In [5]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()
        
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    print(accepted_scores)
    
    return training_data

In [6]:
training_data = model_data_preparation()

[83.0, 76.0, 62.0, 71.0, 65.0, 61.0, 72.0, 86.0, 86.0, 72.0, 80.0, 85.0, 76.0, 61.0, 69.0, 66.0, 71.0, 73.0, 63.0, 75.0, 75.0, 69.0, 73.0, 78.0, 64.0, 61.0, 70.0, 85.0, 65.0, 69.0, 62.0, 71.0, 64.0, 68.0, 60.0, 63.0, 62.0, 100.0, 63.0, 63.0, 66.0, 72.0, 78.0, 71.0, 68.0, 62.0, 89.0, 69.0, 62.0, 64.0, 73.0, 66.0, 67.0, 104.0, 60.0, 73.0, 63.0, 60.0, 69.0, 106.0, 70.0, 63.0, 75.0, 63.0, 63.0, 64.0, 81.0, 64.0, 103.0, 76.0, 70.0, 72.0, 67.0, 90.0, 76.0, 60.0, 73.0, 81.0, 66.0, 69.0, 68.0, 65.0, 67.0, 66.0, 88.0, 63.0, 60.0, 65.0, 65.0, 63.0, 64.0, 84.0, 67.0, 60.0, 63.0, 60.0, 65.0, 71.0, 63.0, 67.0, 62.0, 65.0, 63.0, 64.0, 68.0, 74.0, 63.0, 63.0, 61.0, 61.0, 63.0, 60.0, 65.0, 68.0, 75.0, 61.0, 60.0, 73.0, 60.0, 62.0, 62.0, 87.0, 96.0, 69.0, 82.0, 97.0, 70.0, 101.0, 74.0, 64.0, 68.0, 94.0, 74.0, 61.0, 60.0, 60.0, 60.0, 83.0, 93.0, 80.0, 77.0, 72.0, 86.0, 61.0, 66.0, 71.0, 87.0, 67.0, 81.0, 62.0, 70.0, 61.0, 72.0, 70.0, 79.0]


<ipython-input-5-243cd971d65c>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data_save = np.array(training_data)


In [7]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

In [8]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model

In [9]:
trained_model = train_model(training_data)

Epoch 1/10
338/338 [==============================] - 0s 882us/step - loss: 0.2480
Epoch 2/10
338/338 [==============================] - 0s 870us/step - loss: 0.2348
Epoch 3/10
338/338 [==============================] - 0s 935us/step - loss: 0.2335
Epoch 4/10
338/338 [==============================] - 0s 972us/step - loss: 0.2332
Epoch 5/10
338/338 [==============================] - 0s 867us/step - loss: 0.2325
Epoch 6/10
338/338 [==============================] - 0s 883us/step - loss: 0.2325
Epoch 7/10
338/338 [==============================] - 0s 950us/step - loss: 0.2317
Epoch 8/10
338/338 [==============================] - 0s 938us/step - loss: 0.2317
Epoch 9/10
338/338 [==============================] - 0s 911us/step - loss: 0.2313
Epoch 10/10
338/338 [==============================] - 0s 873us/step - loss: 0.2311


In [ ]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))